### Quantile and Bucket Analysis

In [1]:
import pandas as pd
import numpy as np

In [2]:
frame = pd.DataFrame({"data1":np.random.standard_normal(1000),
                     "data2":np.random.standard_normal(1000)})
frame.head()

,data1,data2
0,-0.717470,2.148111
1,-0.494793,-1.185993
2,-0.287198,1.752414
3,-0.231745,-1.491452
4,1.201763,0.282347


In [3]:
quartiles = pd.cut(frame["data1"], 4)
quartiles.head()

0    (-1.504, 0.132]
1    (-1.504, 0.132]
2    (-1.504, 0.132]
3    (-1.504, 0.132]
4     (0.132, 1.769]
Name: data1, dtype: category
Categories (4, interval[float64, right]): [(-3.147, -1.504] < (-1.504, 0.132] < (0.132, 1.769] < (1.769, 3.405]]

In [4]:
def get_stats(group):
    return pd.DataFrame({"min":group.min(), 
                        "max": group.max(), 
                        "count":group.count(), 
                        "mean":group.mean()})

In [5]:
grouped = frame.groupby(quartiles)

In [6]:
grouped.apply(get_stats)

min       max  count      mean
data1                                                      
(-3.147, -1.504] data1 -3.140594 -1.506111     85 -2.001679
                 data2 -3.592619  2.201836     85  0.009790
(-1.504, 0.132]  data1 -1.499486  0.131951    473 -0.564069
                 data2 -3.150702  3.014605    473  0.020517
(0.132, 1.769]   data1  0.133354  1.766820    402  0.773765
                 data2 -3.595459  2.863243    402  0.004160
(1.769, 3.405]   data1  1.775058  3.405191     40  2.142920
                 data2 -1.603448  3.192858     40  0.360224

In [7]:
grouped.agg(["min", "max", "count", "mean", "std"])

data1                                         data2  \
                       min       max count      mean       std       min   
data1                                                                      
(-3.147, -1.504] -3.140594 -1.506111    85 -2.001679  0.393571 -3.592619   
(-1.504, 0.132]  -1.499486  0.131951   473 -0.564069  0.444691 -3.150702   
(0.132, 1.769]    0.133354  1.766820   402  0.773765  0.426728 -3.595459   
(1.769, 3.405]    1.775058  3.405191    40  2.142920  0.393369 -1.603448   

                                                      
                       max count      mean       std  
data1                                                 
(-3.147, -1.504]  2.201836    85  0.009790  1.044881  
(-1.504, 0.132]   3.014605   473  0.020517  1.038323  
(0.132, 1.769]    2.863243   402  0.004160  0.971199  
(1.769, 3.405]    3.192858    40  0.360224  1.036921

### Filling missing value with group specific value

In [9]:
s = pd.Series(np.random.standard_normal(6))
s

0   -0.936578
1   -1.685519
2    0.265412
3   -0.102411
4   -0.652053
5    1.115336
dtype: float64

In [10]:
s[::2] = np.nan
s

0         NaN
1   -1.685519
2         NaN
3   -0.102411
4         NaN
5    1.115336
dtype: float64

In [11]:
s.fillna(s.mean())

0   -0.224198
1   -1.685519
2   -0.224198
3   -0.102411
4   -0.224198
5    1.115336
dtype: float64

In [13]:
states = ["Ohio", "New York", "Vermont", "Florida","Oregon", "Nevada", "California", "Idaho"]
group_key = ["East", "East", "East", "East","West", "West", "West", "West"]

In [15]:
data = pd.Series(np.random.standard_normal(8), index=states)
data

Ohio          0.099243
New York      0.019836
Vermont      -0.398099
Florida      -0.025997
Oregon       -0.271924
Nevada       -0.250598
California    0.222196
Idaho        -0.242577
dtype: float64

In [16]:
data[["Vermont", "Nevada", "Idaho"]] = np.nan
data

Ohio          0.099243
New York      0.019836
Vermont            NaN
Florida      -0.025997
Oregon       -0.271924
Nevada             NaN
California    0.222196
Idaho              NaN
dtype: float64

In [17]:
data.groupby(group_key).size()

East    4
West    4
dtype: int64

In [18]:
grouped = data.groupby(group_key)

In [19]:
grouped.agg(["size", "count", "mean"])

,size,count,mean
East,4,3,0.031027
West,4,2,-0.024864


In [20]:
def fill_mean(group):
    return group.fillna(group.mean())

In [21]:
grouped.apply(fill_mean)

East  Ohio          0.099243
      New York      0.019836
      Vermont       0.031027
      Florida      -0.025997
West  Oregon       -0.271924
      Nevada       -0.024864
      California    0.222196
      Idaho        -0.024864
dtype: float64

### Example - Group Weighted average and correlation

In [22]:
df = pd.DataFrame({"category":["a", "a", "a", "a","b", "b", "b", "b"], 
                  "data":np.random.standard_normal(8),
                  "weight":np.random.uniform(8)})
df

,category,data,weight
0,a,0.794846,4.297433
1,a,0.319875,4.297433
2,a,-1.640335,4.297433
3,a,1.321277,4.297433
4,b,0.571866,4.297433
5,b,-0.933603,4.297433
6,b,0.906801,4.297433
7,b,1.230701,4.297433


In [24]:
grouped = df.groupby("category")

In [27]:
def get_wavg(group):
    return np.average(group["data"], weights=group["weight"])

In [29]:
grouped.apply(get_wavg) #Weighted average by category

category
a    0.198916
b    0.443941
dtype: float64

In [34]:
close_px = pd.read_csv("stock_px.csv", parse_dates=True, index_col=0)

In [35]:
close_px.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2214 entries, 2003-01-02 to 2011-10-14
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   AAPL    2214 non-null   float64
 1   MSFT    2214 non-null   float64
 2   XOM     2214 non-null   float64
 3   SPX     2214 non-null   float64
dtypes: float64(4)
memory usage: 86.5 KB


In [36]:
def spx_corr(group):
    return group.corrwith(group["SPX"])

In [37]:
rets = close_px.pct_change().dropna()

In [40]:
def get_year(x):
    return x.year

In [41]:
by_year = rets.groupby(get_year)

In [42]:
by_year.apply(spx_corr)

,AAPL,MSFT,XOM,SPX
2003,0.541124,0.745174,0.661265,1.0
2004,0.374283,0.588531,0.557742,1.0
2005,0.467540,0.562374,0.631010,1.0
2006,0.428267,0.406126,0.518514,1.0
2007,0.508118,0.658770,0.786264,1.0
2008,0.681434,0.804626,0.828303,1.0
2009,0.707103,0.654902,0.797921,1.0
2010,0.710105,0.730118,0.839057,1.0
2011,0.691931,0.800996,0.859975,1.0


## Group Transformation and "Unwrapped" GroupBys

In [44]:
df =pd.DataFrame({"key":["a","b","c"]*4, 
                 "value":np.arange(12.)})
df

,key,value
0,a,0.0
1,b,1.0
2,c,2.0
3,a,3.0
4,b,4.0
5,c,5.0
6,a,6.0
7,b,7.0
8,c,8.0
9,a,9.0


In [45]:
g = df.groupby("key")["value"]

In [46]:
g.mean()

key
a    4.5
b    5.5
c    6.5
Name: value, dtype: float64

In [47]:
def get_mean(group):
    return group.mean()

In [48]:
g.transform(get_mean)

0     4.5
1     5.5
2     6.5
3     4.5
4     5.5
5     6.5
6     4.5
7     5.5
8     6.5
9     4.5
10    5.5
11    6.5
Name: value, dtype: float64

In [49]:
g.transform("mean")

0     4.5
1     5.5
2     6.5
3     4.5
4     5.5
5     6.5
6     4.5
7     5.5
8     6.5
9     4.5
10    5.5
11    6.5
Name: value, dtype: float64

In [50]:
def times_two(x):
    return x*2

In [51]:
g.transform(times_two)

0      0.0
1      2.0
2      4.0
3      6.0
4      8.0
5     10.0
6     12.0
7     14.0
8     16.0
9     18.0
10    20.0
11    22.0
Name: value, dtype: float64

In [52]:
def get_rank(group):
    return group.rank(ascending = False)

In [54]:
g.transform(get_rank)

0     4.0
1     4.0
2     4.0
3     3.0
4     3.0
5     3.0
6     2.0
7     2.0
8     2.0
9     1.0
10    1.0
11    1.0
Name: value, dtype: float64

In [55]:
def normalize(group):
    return (group - group.mean())/group.std()

In [56]:
g.transform(normalize)

0    -1.161895
1    -1.161895
2    -1.161895
3    -0.387298
4    -0.387298
5    -0.387298
6     0.387298
7     0.387298
8     0.387298
9     1.161895
10    1.161895
11    1.161895
Name: value, dtype: float64

In [57]:
g.apply(normalize)

key    
a    0    -1.161895
     3    -0.387298
     6     0.387298
     9     1.161895
b    1    -1.161895
     4    -0.387298
     7     0.387298
     10    1.161895
c    2    -1.161895
     5    -0.387298
     8     0.387298
     11    1.161895
Name: value, dtype: float64

In [62]:
g.transform("mean")

0     4.5
1     5.5
2     6.5
3     4.5
4     5.5
5     6.5
6     4.5
7     5.5
8     6.5
9     4.5
10    5.5
11    6.5
Name: value, dtype: float64

In [66]:
#Here, we are doing arithmetic between the outputs of multiple GroupBy operations
#instead of writing a function and passing it to groupby(...).apply. That is what is
#meant by “unwrapped.”

normalize = (df['value'] - g.transform('mean'))/g.transform("std")

In [67]:
normalize

0    -1.161895
1    -1.161895
2    -1.161895
3    -0.387298
4    -0.387298
5    -0.387298
6     0.387298
7     0.387298
8     0.387298
9     1.161895
10    1.161895
11    1.161895
Name: value, dtype: float64

### Pivot tables and crosstab

In [92]:
tips = pd.read_csv("tips.csv")
tips = tips.drop(['sex','time'], axis=1)
tips["tip_pct"] = tips["tip"]/tips["total_bill"]

In [93]:
tips.head()

,total_bill,tip,smoker,day,size,tip_pct
0,16.99,1.01,No,Sun,2,0.059447
1,10.34,1.66,No,Sun,3,0.160542
2,21.01,3.50,No,Sun,3,0.166587
3,23.68,3.31,No,Sun,2,0.139780
4,24.59,3.61,No,Sun,4,0.146808


In [94]:
tips.pivot_table(index=["day","smoker"]) #mean default

size       tip   tip_pct  total_bill
day  smoker                                          
Fri  No      2.250000  2.812500  0.151650   18.420000
     Yes     2.066667  2.714000  0.174783   16.813333
Sat  No      2.555556  3.102889  0.158048   19.661778
     Yes     2.476190  2.875476  0.147906   21.276667
Sun  No      2.929825  3.167895  0.160113   20.506667
     Yes     2.578947  3.516842  0.187250   24.120000
Thur No      2.488889  2.673778  0.160298   17.113111
     Yes     2.352941  3.030000  0.163863   19.190588

In [95]:
tips.pivot_table(index=["day"], columns="smoker", values=["size"])

size          
smoker        No       Yes
day                       
Fri     2.250000  2.066667
Sat     2.555556  2.476190
Sun     2.929825  2.578947
Thur    2.488889  2.352941

In [96]:
tips.pivot_table(index=["day"], columns="smoker", values=["size"], margins=True)
#All row and column labels

size                    
smoker        No       Yes       All
day                                 
Fri     2.250000  2.066667  2.105263
Sat     2.555556  2.476190  2.517241
Sun     2.929825  2.578947  2.842105
Thur    2.488889  2.352941  2.451613
All     2.668874  2.408602  2.569672

In [97]:
tips.pivot_table(index=["day"], columns="smoker", values=["size"], margins=True, aggfunc=len)

size         
smoker   No Yes  All
day                 
Fri       4  15   19
Sat      45  42   87
Sun      57  19   76
Thur     45  17   62
All     151  93  244

In [101]:
tips.pivot_table(index=["size", "smoker"], columns="day", values=["tip_pct"], margins=True, fill_value=0)

tip_pct                                        
day               Fri       Sat       Sun      Thur       All
size smoker                                                  
1    No      0.000000  0.137931  0.000000  0.181728  0.159829
     Yes     0.223776  0.325733  0.000000  0.000000  0.274755
2    No      0.139622  0.162705  0.168859  0.165827  0.164996
     Yes     0.175402  0.148668  0.207893  0.158843  0.166706
3    No      0.187735  0.154661  0.152663  0.084246  0.149671
     Yes     0.000000  0.144995  0.152660  0.204952  0.157543
4    No      0.000000  0.150096  0.148143  0.138919  0.147604
     Yes     0.117750  0.124515  0.193370  0.155410  0.142036
5    No      0.000000  0.000000  0.206928  0.121389  0.178415
     Yes     0.000000  0.106572  0.065660  0.000000  0.086116
6    No      0.000000  0.000000  0.103799  0.173706  0.156229
All          0.169913  0.153152  0.166897  0.161276  0.160803

In [102]:
#A cross-tabulation (or crosstab for short) is a special case of a pivot table that com‐
#putes group frequencies

In [103]:
from io import StringIO

In [104]:
data = """Sample Nationality Handedness
 .....: 1 USA Right-handed
 .....: 2 Japan Left-handed
 .....: 3 USA Right-handed
 .....: 4 Japan Right-handed
 .....: 5 Japan Left-handed
 .....: 6 Japan Right-handed
 .....: 7 USA Right-handed
 .....: 8 USA Left-handed
 .....: 9 Japan Right-handed
 .....: 10 USA Right-handed"""

In [105]:
data

'Sample Nationality Handedness\n1 USA Right-handed\n2 Japan Left-handed\n3 USA Right-handed\n4 Japan Right-handed\n5 Japan Left-handed\n6 Japan Right-handed\n7 USA Right-handed\n8 USA Left-handed\n9 Japan Right-handed\n10 USA Right-handed'

In [106]:
data = pd.read_table(StringIO(data),sep="\s+")
data

,Sample,Nationality,Handedness
0,1,USA,Right-handed
1,2,Japan,Left-handed
2,3,USA,Right-handed
3,4,Japan,Right-handed
4,5,Japan,Left-handed
5,6,Japan,Right-handed
6,7,USA,Right-handed
7,8,USA,Left-handed
8,9,Japan,Right-handed
9,10,USA,Right-handed


In [107]:
pd.crosstab(data["Nationality"], data["Handedness"], margins=True)

Handedness,Left-handed,Right-handed,All
Nationality,,,
Japan,2,3,5
USA,1,4,5
All,3,7,10


In [108]:
pd.crosstab(tips["day"], tips["smoker"], margins=True)

smoker,No,Yes,All
day,,,
Fri,4,15,19
Sat,45,42,87
Sun,57,19,76
Thur,45,17,62
All,151,93,244
